# Imports

In [1]:
import calendar
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import imageio.v2 as imageio
import itertools
import matplotlib
import matplotlib.pyplot as plt
import branca.colormap as cm
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [16, 11] 

In [2]:
station_df = pd.read_csv('input_data/all_stations_by_hour_weekday.csv')

In [3]:
station_df = gpd.GeoDataFrame(
    station_df, geometry=gpd.points_from_xy(station_df.coordinatesX, station_df.coordinatesY))
station_df["copy_geometry"] = station_df.geometry


In [4]:
plz_shape_df = gpd.read_file('input_data/plz-5stellig.shp', dtype={'plz': str})
plz_region_df = pd.read_csv(
    'input_data/zuordnung_plz_ort.csv', 
    sep=',', 
    dtype={'plz': str}
)
plz_einwohner_df = pd.read_csv(
    'input_data/plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)

In [5]:
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)


In [6]:
station_df = station_df.set_crs('epsg:4236')

In [ ]:
hamburg_df = germany_df.query('ort == "Hamburg"')

joined_df = hamburg_df.sjoin(station_df, how="left")
agg_df = joined_df.groupby(['plz', 'resultHour', 'resultWeekday']).agg({"average_res": sum, "thingID": "nunique", "einwohner": "max"}).reset_index()
agg_df["average_res_pop"] = agg_df["average_res"]/ agg_df["einwohner"]
combined_df = hamburg_df.merge(agg_df, how='left', on="plz")

/Users/felixude/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/geopandas/geodataframe.py:2090: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4236

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


# Creating Station Hour Plots

In [152]:
from tqdm import tqdm
hamburg_df = germany_df.query('ort == "Hamburg"')


for day, hour in tqdm(list(itertools.product(range(1, 8, 1), range(0, 24, 1)))):

    unique_coords = station_df.query(f'resultHour == {hour} and resultWeekday == {day}')
    plt.ion()

    hamburg = hamburg_df.plot(
    )

    cur = unique_coords.plot(ax=hamburg,
                       column='average_res',
                       cmap='plasma', 
                             vmin = 1,
                             vmax = 17,
                             legend_kwds={"label": "Bikes"}


                      )

    cur.set_xlim(9.7, 10.37)
    cur.set_ylim(53.38, 53.75)
    cur.grid(False)
    cur.set_facecolor("lavender")
    w_day = calendar.day_name[day-1]
    if hour < 10:
        hour_string = "0" + str(hour)
    else:
        hour_string = str(hour)
    cur.set_title(f"Hamburg bike stations at {w_day} {hour_string}:00", fontsize=20)
    cur.tick_params(left = False, right = False , labelleft = False ,
            labelbottom = False, bottom = False)
    file_name = f"images/stations_{day}_{hour}.jpg"
    cur.figure.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close(cur.figure)




100%|███████████████████████████████████████████████████████████████| 168/168 [00:56<00:00,  2.95it/s]


In [133]:
combined_df["average_res_pop"].max()

0.0658504206742643

In [151]:
from tqdm import tqdm
for day, hour in tqdm(list(itertools.product(range(1, 8, 1), range(0, 24, 1)))):
    unique_coords = combined_df.query(f'resultHour == {hour} and resultWeekday == {day}')
    fig, ax = plt.subplots()
    plt.ion()
    cur = hamburg_df.plot(ax=ax, color ='grey')
    cur = unique_coords.plot(
                       ax=ax,
                       column='average_res_pop',
                       cmap='plasma', 
                       legend=True,
                       norm=plt.Normalize(vmin=0.0, vmax=0.01),
                        legend_kwds={"label": "Bikes per inhabitant"},
                        vmax=0.01

                      )


    ax.set_xlim(9.7, 10.37)
    ax.set_ylim(53.38, 53.75)
    ax.grid(False)
    ax.set_facecolor("lavender")
    w_day = calendar.day_name[day-1]
    if hour < 10:
        hour_string = "0" + str(hour)
    else:
        hour_string = str(hour)
    ax.set_title(f"Hamburg bikes per distict at {w_day} {hour_string}:00")
    cur.tick_params(left = False, right = False , labelleft = False ,
            labelbottom = False, bottom = False)
    file_name = f"images/zip_{day}_{hour}.jpg"
    fig.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close(fig)



100%|███████████████████████████████████████████████████████████████| 168/168 [01:01<00:00,  2.73it/s]


# Graph Calculation

In [13]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import folium
import osmnx as ox
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from IPython.display import IFrame
from folium.plugins import MarkerCluster
ox.config(log_console=True, use_cache=True)
import json

/Users/felixude/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [14]:
class DictSmallest(dict):
    def __setitem__(self, key, value):
        if (key not in self) or (key in self and self[key] > value):
            dict.__setitem__(self, key, value)
    def update(self, dict):
        for key, value in dict.items():
            if (key not in self) or (key in self and self[key] > value):
                self[key] =  value
        

In [156]:
unique_station = station_df.groupby('thingID')[["coordinatesY", "coordinatesX"]].min()
unique_station = unique_station.reset_index(drop=True)

all_sub_station = pd.read_csv('input_data/cleaned_stations.csv')

In [157]:
all_sub_station = all_sub_station.rename({"lat": "coordinatesY", "lon": "coordinatesX"}, axis=1)


In [158]:
all_sub_station.shape

(2301, 6)

In [184]:
all_sub_station.head()

,id,name,coordinatesY,coordinatesX,subway,suburban
0,618002549,Hamburg Hbf,53.551753,10.005659,False,False
1,618098549,Hamburg Hbf (S-Bahn),53.551753,10.005659,False,False
2,670694826,"Hauptbahnhof Süd, Hamburg",53.551753,10.005659,False,False
3,690694827,"Hauptbahnhof Nord, Hamburg",53.551753,10.005659,False,False
4,694785,"Mönckebergstraße, Hamburg",53.551312,10.002019,True,False


In [213]:
def create_coords_dict(poi_df, center_location ="Hamburg, Germany",  dist=1000):
    G =  ox.graph_from_address(center_location, dist=dist, network_type="walk", simplify=True)
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    list_of_poi = []
    for index, row in poi_df.iterrows():
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))

    G = ox.project_graph(G)

    node_distances = DictSmallest()

    for poi in tqdm(list_of_poi):
        tmp_res = nx.shortest_path_length(G, source=poi)
        node_distances.update(tmp_res)
        
    
 

    coords = key: {"lon": G.nodes[key]["lon"], "lat": G.nodes[key]["lat"], "distance": node_distances[key]} for key in list(G.nodes())}
    return coords

In [215]:
node_distances = DictSmallest()

for poi in tqdm(list_of_poi):
    tmp_res = nx.shortest_path_length(G, source=poi)
    node_distances.update(tmp_res)




coords = {key: {"lon": G.nodes[key]["x"], "lat": G.nodes[key]["y"], "distance": node_distances[key]} for key in list(G.nodes())}

100%|████████████████████████████████████████████████████████████| 2301/2301 [00:07<00:00, 313.04it/s]


In [35]:
coords = create_coords_dict(unique_station)

100%|█████████████████████████████████████████████████████████████████| 288/288 [00:00<00:00, 318.03it/s]


In [37]:

coords_df = gpd.GeoDataFrame(coords)
coords_df.geometry = gpd.points_from_xy(coords_df.lon, coords_df.lat)

In [82]:
coords_df.shape

(3062, 4)

In [92]:
hamburg_df.shape

(101, 10)

In [79]:
combined_df = gpd.sjoin(coords_df, hamburg_df,  how="left")

/var/folders/s1/dsr96z496qjg8hh8g1nn0x7m0000gn/T/ipykernel_41046/2000251557.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  combined_df = gpd.sjoin(coords_df, hamburg_df,  how="left")


In [104]:
combined_df["n_nodes_plz"]= combined_df.groupby('plz')["geometry"].transform("count")

In [106]:
combined_df["pop_perc"] = combined_df["einwohner"] / combined_df["n_nodes_plz"]

In [108]:
combined_df["sum_pop_distance"] = combined_df["distance"] * combined_df["pop_perc"]

In [111]:
combined_df["sum_pop_distance"].sum() / combined_df["pop_perc"].sum()

5.417458808947274

In [115]:
combined_df.groupby("plz")["distance"].mean()

plz
20095    5.621827
20097    5.420000
20099    7.258503
20354    5.648000
20355    3.261905
20457    5.338010
20459    4.271429
Name: distance, dtype: float64

In [87]:
final_df = pd.merge(combined_df, agg_df, on="plz", how="left")

In [120]:
final_df.groupby(["plz", "resultHour" ,"resultWeekday"])["average_res"].mean().reset_index()

,plz,resultHour,resultWeekday,average_res
0,20095,0.0,1.0,39.003720
1,20095,0.0,2.0,35.963449
2,20095,0.0,3.0,32.125469
3,20095,0.0,4.0,34.123018
4,20095,0.0,5.0,34.832975
...,...,...,...,...
1171,20459,23.0,3.0,33.098056
1172,20459,23.0,4.0,33.418238
1173,20459,23.0,5.0,31.568043
1174,20459,23.0,6.0,31.860257


# Visualization

In [20]:
trip_times = range(1, 51, 1)

iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 51, 5))
    

In [21]:
colormap

In [178]:
G = ox.graph_from_address("Hamburg, Germany", dist=1000, network_type="walk", simplify=True)

In [189]:
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
m = folium.Map(
    location=[53.555, 9.9914],
    zoom_start=15,
    prefer_canvas=True,
)



    
list_of_poi = []

for index, row in all_sub_station.iterrows():
    list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))



In [219]:
node_distances = DictSmallest()

for poi in tqdm(list_of_poi):
    tmp_res = nx.shortest_path_length(G, source=poi)
    node_distances.update(tmp_res)

100%|████████████████████████████████████████████████████████████| 2301/2301 [00:07<00:00, 313.99it/s]


In [ ]:
tmp_res

In [220]:
  def color_mapping_function(val, iso_colors):

        for time, color in zip(trip_times, iso_colors):
            if val < time :
                return color

        return iso_colors[-1]

coords = {key: {"x": G.nodes[key]["x"], "y": G.nodes[key]["y"], "color": color_mapping_function(node_distances[key], iso_colors)} for key in list(G.nodes())}

In [224]:
colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 51, 5))

import folium
m = folium.Map(location=[53.555, 9.9914]
, zoom_start=12,prefer_canvas=True, 
               )



for val in coords.values():
    folium.Circle(
      location=[val["y"],val["x"]],
        radius=50,
      #popup="Test",
    stroke=False,
    fill=True,
    color = val["color"],
    fill_opacity=0.3,
    interactive=True

   ).add_to(m)
colormap.add_to(m)
m

In [217]:
coords

{8083334: {'lon': 9.9943664, 'lat': 53.5587088, 'distance': 3},
 11206918: {'lon': 10.0145505, 'lat': 53.5481288, 'distance': 4},
 13777942: {'lon': 9.9945686, 'lat': 53.5504802, 'distance': 2},
 13777944: {'lon': 10.0042371, 'lat': 53.5523938, 'distance': 1},
 13804884: {'lon': 9.9894914, 'lat': 53.5508123, 'distance': 6},
 13804885: {'lon': 9.9891662, 'lat': 53.5510671, 'distance': 7},
 13870878: {'lon': 10.0069912, 'lat': 53.5543374, 'distance': 6},
 13870880: {'lon': 10.0063504, 'lat': 53.5548102, 'distance': 8},
 13879893: {'lon': 10.002711, 'lat': 53.5533705, 'distance': 2},
 13879900: {'lon': 10.0038931, 'lat': 53.5538144, 'distance': 2},
 13879907: {'lon': 10.0038483, 'lat': 53.5541033, 'distance': 7},
 13892002: {'lon': 10.0005059, 'lat': 53.5556995, 'distance': 0},
 13892004: {'lon': 10.0009145, 'lat': 53.5556786, 'distance': 6},
 13951247: {'lon': 9.9993582, 'lat': 53.5521267, 'distance': 5},
 13951251: {'lon': 10.0025086, 'lat': 53.5534776, 'distance': 3},
 13951254: {'lon'

In [180]:
%%time

trip_times = range(1, 51, 1)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def create_coords_dict(poi_df, filename, center_location ="Hamburg, Germany",  dist=5000):
    G =  ox.graph_from_address(center_location, dist=dist, network_type="walk", simplify=True)
    # 2 - Create nodes geodataframe from Graph network (G)
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)

    list_of_poi = []

    for index, row in poi_df.iterrows():
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))

    G = ox.project_graph(G)

    node_distances = DictSmallest()

    for poi in tqdm(list_of_poi):
        tmp_res = nx.shortest_path_length(G, source=poi)
        node_distances.update(tmp_res)
        
    
    def color_mapping_function(val, iso_colors):

        for time, color in zip(trip_times, iso_colors):
            if val < time :
                return color

        return iso_colors[-1]

    coords = {key: {"x": G.nodes[key]["lon"], "y": G.nodes[key]["lat"], "color": color_mapping_function(node_distances[key])} for key in list(G.nodes())}
    with open(f'coords/{filename}.json', 'w') as outfile:
        json.dump(coords, outfile, indent=4)
  

    

def create_folium_plot(coord_file, file_name, caption):

    with open("coords/" + coord_file, "r") as f:
        coords = json.loads(f.read())


    colormap = cm.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=range(0, 51, 5))
    colormap.caption = caption
   
    import folium
    m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True, 
                   )



    for val in coords.values():
        folium.Circle(
          location=[val["y"],val["x"]],
            radius=50,
          #popup="Test",
        stroke=False,
        fill=True,
        color = val["color"],
        fill_opacity=0.3,
        interactive=True

       ).add_to(m)
    colormap.add_to(m)
    m.save(f'pages/maps/{file_name}.html')

CPU times: user 5.52 ms, sys: 620 µs, total: 6.14 ms
Wall time: 5.46 ms


In [ ]:
create_coords_dict(unique_station, "station_10000", dist=10000)

In [ ]:
create_coords_dict(all_sub_station.loc[(all_sub_station["subway"] == True) | (all_sub_station["suburban"] == True)], "all_sub_10000", dist=10000)

In [167]:
%%time
create_coords_dict(all_sub_station, "all_hvv_10000", dist=10000)

100%|█████████████████████████████████████████████████████████████| 2301/2301 [06:12<00:00,  6.18it/s]


CPU times: user 24min 5s, sys: 4.75 s, total: 24min 10s
Wall time: 24min 9s


In [28]:
%%time
caption_bike = "Walking distance in minutes to nearest bike station"
create_folium_plot("bike_10000.json", "hamburg_bike_darker_10000",  caption=caption_bike)

CPU times: user 28.2 s, sys: 672 ms, total: 28.9 s
Wall time: 28.8 s


In [29]:
%%time
caption_bike = "Walking distance in minutes to nearest subway/-urban station"
create_folium_plot("all_sub_10000.json","hamburg_hvv_darker_10000", caption=caption_bike)

CPU times: user 28.3 s, sys: 493 ms, total: 28.8 s
Wall time: 28.7 s


In [ ]:
all_poi = pd.concat([all_sub_station[["coordinatesY", "coordinatesX"]], unique_station])

In [ ]:
create_coords_dict(all_poi, "all_poi_10000", dist=10000)

In [31]:
%%time
caption_bike = "Walking distance in minutes to nearest subway/-urban or bike station"
create_folium_plot("all_poi_10000.json", "hamburg_all_darker_10000", caption=caption_bike)

CPU times: user 28.4 s, sys: 638 ms, total: 29.1 s
Wall time: 28.9 s
